In [ ]:
# If your container doesn't have any of the following resources, just run this cell once.
!pip install pymysql
!pip install sqlalchemy
!pip install gensim
import nltk
nltk.download('punkt')

# Exploring the MIMIC Chest X-ray reports

In this notebook, we will take a glance at the MIMIC dataset. Then narrow our scope to the chest x-ray reports. With a few examples of how to interact with MySQL database to pull the data.

In [ ]:
# import packages
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob
import re
import scipy
import matplotlib.pyplot as plt
import ipywidgets
from IPython.display import display, HTML, clear_output
from ipywidgets import interact, interactive, fixed
from sqlalchemy import create_engine
from gensim.parsing.preprocessing import STOPWORDS
from collections import Counter
import sys
%matplotlib inline

### 1. Connect MySQL DB
Set up our MySQL connection with SQL Alchemy (this helps us to read directly into Pandas DataFrames

In [ ]:
engine = create_engine('mysql+pymysql://{0}:{1}@{2}:{3}/{4}'
                      .format('jovyan', 
                             getpass.getpass('Enter MySQL password for jovyan'),
                             'mysql',
                             '3306',
                             'mimic2'))

### 2. Explore the MIMIC
Before we move ahead, we will do some counts of patients, admissions and notes to ensure connectivity and also get a sense of the dataset

In [ ]:
pd.read_sql_query('SELECT count(*) as PatientCount from d_patients', engine)
pd.read_sql_query('SELECT count(*) as AdmissionCount from admissions', engine)
pd.read_sql_query('SELECT count(*) as NoteCount from noteevents', engine)
pd.read_sql_query('SELECT count(DISTINCT code) as Distinct_ICD9_Codes from icd9 LIMIT 5', engine)

In [ ]:
#Let try to pull the most popular ICD9 codes:
pd.read_sql_query("""SELECT 
                                 code, description, COUNT(*) counts 
                             FROM 
                                 icd9 
                             GROUP BY 
                                 code,description 
                             ORDER BY counts DESC 
                             LIMIT 10""", engine)

In [ ]:
#Get down to see how to pull notes:
pd.read_sql_query('SELECT * from noteevents LIMIT 5', engine)

### now let's get a dataframe of Patient/Admit/Pneumonia
#### This is a useful and efficient query.

In [ ]:

pneumonia_query = """
    SELECT 
a.subject_id
,a.hadm_id
,a.admit_dt
,(CASE WHEN pneu.HADM_ID IS NOT NULL THEN 1 ELSE 0 END) as Encounter_Pneumonia_Diagnosis
FROM admissions a
LEFT JOIN 
(
    SELECT
    d.HADM_ID
    FROM  icd9 d
    WHERE
        (code like '486%%')
    GROUP BY d.HADM_ID
) pneu
ON a.HADM_ID = pneu.HADM_ID
"""
pat_admit_pneumonia_df = pd.read_sql_query(pneumonia_query, engine)
pat_admit_pneumonia_df.head(10)

In [ ]:
pat_admit_pneumonia_df.shape

In [ ]:
# let's get a count of how many PNEUMONIA vs NO-PNEUMONIA admits we have
pneumonia_admit_count_df = pat_admit_pneumonia_df.groupby('Encounter_Pneumonia_Diagnosis').size()
pneumonia_admit_count_df

## 4. What Radiology Reports Are We Interested In?

### Use this  SQL queries to explore all readiology reports and find the sub-categories you want to include


```SQL
SELECT
subject_id
,hadm_id
,LTRIM(RTRIM(text)) as txt
FROM noteevents
WHERE category = 'RADIOLOGY_REPORT'
    AND subject_id is not NULL
    AND hadm_id is not NULL        
GROUP BY subject_id, hadm_id, txt
```


In [ ]:
# now let's pull a frame of all the FIRST (sorted by text which begins with date) CHEST X-RAY notes
chest_xray_note_query = \
"""
SELECT
subject_id
,hadm_id
,LTRIM(RTRIM(text)) as txt
FROM noteevents
WHERE category = 'RADIOLOGY_REPORT'
    AND subject_id is not NULL
    AND hadm_id is not NULL        
GROUP BY subject_id, hadm_id, txt
"""
chest_xray_note_df = pd.read_sql_query(chest_xray_note_query, engine)
print("Shape of DataFrame Queries", chest_xray_note_df.shape)
chest_xray_note_df.head(10)

In [ ]:
for _,row in chest_xray_note_df.sample(frac=0.2).iterrows():
    clear_output()
    print(row['txt'])
    input('continue')

### Modify the SQL query to filter radiology reports to "only" include the type of reports we are interested in

**HINT:** Use the SQL [`LIKE`](https://www.w3schools.com/sql/sql_like.asp) statement


```SQL
SELECT
subject_id
,hadm_id
,LTRIM(RTRIM(text)) as txt
FROM noteevents
WHERE category = 'RADIOLOGY_REPORT'
    AND subject_id is not NULL
    AND hadm_id is not NULL        
GROUP BY subject_id, hadm_id, txt
```


In [ ]:
chest_xray_note_query = \
"""

"""
chest_xray_note_df = pd.read_sql_query(chest_xray_note_query, engine)
print("Shape of DataFrame Queries", chest_xray_note_df.shape)
chest_xray_note_df.head(10)

## Use Pandas [`merge`](https://pandas.pydata.org/pandas-docs/stable/merging.html) function to merge our DataFrame that has ICD-9 codes with the DataFrame that has notes 

### Exercise: What should be the value for the keyword argument `on`?

In [ ]:
pneumonia_note_df = pd.merge(pat_admit_pneumonia_df, chest_xray_note_df, on = ???)
pneumonia_note_df.head()

In [ ]:
pneumonia_note_df.tail()

## 5. Some ideas on selecting which reports to use 
![image.png](./Slide_pngs/Slide16.png)

## 6. What sample size should you use?
![image.png](./Slide_pngs/Slide18.png)

## 7. Example Code For Stratified Sampling


We sampled notes where the encounter was coded for Pneumonia (ICD-9 code 486.* ) and where it was not coded.  We performed stratified sampling of a percentage of notes from one and the remainder from the other.

In [ ]:
# select notes from the encounters without code
group0_notes=pneumonia_note_df.loc[pneumonia_note_df['Encounter_Pneumonia_Diagnosis'] == 0]
# select notes from the encounters with code
group1_notes=pneumonia_note_df.loc[pneumonia_note_df['Encounter_Pneumonia_Diagnosis'] == 1]
# sampling within each group and then shuffle
samples=pd.concat([group0_notes.sample(10),group1_notes.sample(30)]).sample(frac=1)
samples.shape

In [ ]:
# display the first 10 notes
samples.head(10)

<br/><br/>This material presented as part of the DeCART Data Science for the Health Science Summer Program at the University of Utah in 2018.<br/>
Presenters : Dr. Wendy Chapman, Jianlin Shi <br> Acknowledgement: Many thanks to Kelly Peterson and Olga Patterson because part of the materials are adopted from his previous work.